In [1]:
import sys

sys.path.append(
    "C:\\Users\\Wyss User\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages"
)

import biolib
import gzip
import math
import os
import requests

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

from io import StringIO
from IPython.display import Image
from matplotlib_venn import venn2
from scipy import stats
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

c:\Users\Wyss User\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


2023-12-04 15:47:50,229 | INFO : Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-12-04 15:47:50,232 | INFO : NumExpr defaulting to 8 threads.


In [8]:
ALL_FASTA_OUTPUT_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\all_fasta.fasta"
ALL_TARGETS_OUTPUT_DIRECTORY = (
    "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\tmhmm_all_targets"
)

ASSAY_LIST_PATH = (
    "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\assay-list-olink-explore-3072.xlsx"
)
BRAIN_RNA_SEQ_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\brain_rna_seq.csv"
HGNC_IDS = (
    "https://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/tsv/hgnc_complete_set.txt"
)
MICROGLIA_FASTA_OUTPUT_PATH = (
    "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\microglia_fasta.fasta"
)
MATURE_ASTROCYTE_FASTA_OUTPUT_PATH = (
    "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\mature_astrocyte_fasta.fasta"
)
OLIGODENDROCYTE_FASTA_OUTPUT_PATH = (
    "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\oligodendrocyte_fasta.fasta"
)
PLATE_LAYOUT_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\Plate Layout.xlsx"


RAW_DATA_FILE = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\LCSET_28343_10-4-2023_EXTENDED_NPX_2023-10-20.csv"
UNIPROT_TO_FASTA_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\uniprotkb_proteome_UP000005640_2023_11_20.fasta.gz"


PANELS = [
    "Cardiometabolic",
    "Cardiometabolic_II",
    "Inflammation",
    "Inflammation_II",
    "Neurology",
    "Neurology_II",
    "Oncology",
    "Oncology_II",
]


UNIMPORTANT_COLUMNS = [
    "Index",
    "OlinkID",
    "Panel_Lot_Nr",
    "PlateID",
    "WellID",
    "Normalization",
    "Processing_StartDate",
    "Processing_EndDate",
    "AnalyzerID",
    "ExploreVersion",
]


X_AXIS_ORDER = [
    "SEC Fract 6 ",
    "SEC Fract 7",
    "SEC Fract 8",
    "SEC Fract 9",
    "SEC Fract 10",
    "SEC Fract 11",
    "SEC Fract 12",
    "SEC Fract 13",
    "SEC Fract 14",
    "SEC Fract 15",
    "CSF A Internal EV",
    "CSF B Internal EV",
    "CSF C Internal EV",
    "CSF D Internal EV",
    "Plas A Internal EV",
    "Plas B Internal EV",
    "Plas C Internal EV",
    "Plasma D Internal EV",
]

CSF_SAMPLES = [
    "SEC Fract 6 ",
    "SEC Fract 7",
    "SEC Fract 8",
    "SEC Fract 9",
    "SEC Fract 10",
    "SEC Fract 11",
    "SEC Fract 12",
    "SEC Fract 13",
    "SEC Fract 14",
    "SEC Fract 15",
    "CSF A Internal EV",
    "CSF B Internal EV",
    "CSF C Internal EV",
    "CSF D Internal EV",
]

In [6]:
assay_list = pd.read_excel(ASSAY_LIST_PATH)


# map uniprot id's to fasta
def parse_gz_file(file_path):
    protein_dict = {}
    current_uniprot_id = None
    current_sequence = ""

    with gzip.open(file_path, "rt") as f:
        for line in f:
            line = line.strip()

            if line.startswith(">"):
                if current_uniprot_id is not None:
                    protein_dict[current_uniprot_id] = current_sequence
                    current_sequence = ""
                if "|" in line:
                    current_uniprot_id = line.split("|")[1].strip()
                else:
                    print(f"Skipping line without expected format: {line}")
                    current_uniprot_id = None
            else:
                current_sequence += line
        if current_uniprot_id is not None:
            protein_dict[current_uniprot_id] = current_sequence

    return protein_dict


protein_data = parse_gz_file(UNIPROT_TO_FASTA_PATH)

assay_list["Sequence"] = assay_list["UniProt ID"].map(
    lambda x: protein_data.get(x, "N/A")
)

In [15]:
def export_fasta(output_path, df):
    with open(output_path, "w") as output_file:
        for index, row in df.iterrows():
            output_file.write(f'>{row["UniProt ID"]}\n')
            output_file.write(f'{row["Sequence"]}\n')


export_fasta(ALL_FASTA_OUTPUT_PATH, assay_list)

deeptmhmm = biolib.load("DTU/DeepTMHMM")


def tmhmm_localization(targets, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    os.chdir(output_directory)

    with open("query.fasta", "w") as fasta_file:
        for _, row in targets.iterrows():
            sequence = row["Sequence"]
            uniprot_id = row["UniProt ID"]
            fasta_line = f">{uniprot_id}\n{sequence}\n"
            fasta_file.write(fasta_line)
        biolib.utils.STREAM_STDOUT = True  # Stream progress from app in real time
        deeptmhmm_job = deeptmhmm.cli(
            args="--fasta query.fasta", machine="local"
        )  # Blocks until done
        deeptmhmm_job.save_files(output_directory)  # Saves all results to `result` dir


tmhmm_localization(assay_list, ALL_TARGETS_OUTPUT_DIRECTORY)

2023-12-04 19:28:35,461 | INFO : Loaded project DTU/DeepTMHMM:1.0.24
2023-12-04 19:28:37,556 | INFO : Job "e02cb936-e4b9-4eb8-891c-9ff91eab57b7" running...
Running DeepTMHMM on 2915 sequences...
Step 1/4 | Loading transformer model...

Step 2/4 | Generating embeddings for sequences...
Generating embeddings: 100% 2915/2915 [2:44:55<00:00,  3.39s/seq]  

Step 3/4 | Predicting topologies for sequences in batches of 1...
Topology prediction: 100% 2915/2915 [1:37:55<00:00,  2.02s/seq]   

Step 4/4 | Generating output...
2023-12-04 23:52:28,061 | INFO : Saving 3 files to C:\Users\Wyss User\Documents\EVs\OLINK\tmhmm_all_targets...


In [22]:
import gffutils

# Path to your GFF3 file
gff_file_path = (
    "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\tmhmm_all_targets\\TMRs.gff3"
)

# Create a temporary SQLite database
db = gffutils.create_db(
    gff_file_path,
    ":memory:",
    force=True,
    keep_order=True,
    merge_strategy="merge",
    sort_attribute_values=True,
)

# Query and print all features from the temporary database
for feature in db.all_features():
    print(feature)

P16860	signal	1	26	.				
P16860	outside	27	134	.				
//	.	.	.	.	.	.	.	
P19429	inside	1	210	.				
//	.	.	.	.	.	.	.	
P61978	inside	1	463	.				
//	.	.	.	.	.	.	.	
P17676	inside	1	345	.				
//	.	.	.	.	.	.	.	
P08670	inside	1	466	.				
//	.	.	.	.	.	.	.	
O96017	inside	1	543	.				
//	.	.	.	.	.	.	.	
P34998	signal	1	23	.				
P34998	outside	24	117	.				
P34998	TMhelix	118	139	.				
P34998	inside	140	178	.				
P34998	TMhelix	179	203	.				
P34998	outside	204	218	.				
P34998	TMhelix	219	247	.				
P34998	inside	248	254	.				
P34998	TMhelix	255	282	.				
P34998	outside	283	298	.				
P34998	TMhelix	299	324	.				
P34998	inside	325	335	.				
P34998	TMhelix	336	360	.				
P34998	outside	361	367	.				
P34998	TMhelix	368	396	.				
P34998	inside	397	444	.				
//	.	.	.	.	.	.	.	
Q969D9	signal	1	28	.				
Q969D9	outside	29	159	.				
//	.	.	.	.	.	.	.	
P55082	signal	1	21	.				
P55082	outside	22	148	.				
P55082	TMhelix	149	168	.				
P55082	inside	169	362	.				
//	.	.	.	.	.	.	.	
O95988	inside	1	128	.				
//	.	.	.	.	